In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import gradio as gr

# Load Mistral model and tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"  # Ensure the model is available on Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Function to generate response
def generate_response(prompt_txt):
    inputs = tokenizer(prompt_txt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=256, temperature=0.5)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response
